In [1]:
from pyomo.environ import *

In [8]:
# Define sets
I = ['Plant1', 'Plant2', 'Plant3', 'Plant4', 'Plant5', 'Plant6']
T = range(1, 16)

# Sample data
# Startup costs
cu = {'Plant1': 10324, 'Plant2': 5678, 'Plant3': 7802, 'Plant4': 12899, 'Plant5': 4596, 'Plant6': 9076}
# Shutdown costs
cd = {'Plant1': 2673, 'Plant2': 5893, 'Plant3': 982, 'Plant4': 6783, 'Plant5': 2596, 'Plant6': 3561}
# Fixed costs
crf = {'Plant1': 2000, 'Plant2': 3000, 'Plant3': 2500, 'Plant4': 4000, 'Plant5': 3500, 'Plant6': 4500}
# Variable costs (changes for each time step)
crv = {
    'Plant1': [20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35],
    'Plant2': [15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
    'Plant3': [18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32],
    'Plant4': [25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
    'Plant5': [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36],
    'Plant6': [30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44]
}
# Lower and upper bounds for power output
l = {'Plant1': 50, 'Plant2': 40, 'Plant3': 30, 'Plant4': 60, 'Plant5': 55, 'Plant6': 65}
u = {'Plant1': 500, 'Plant2': 600, 'Plant3': 550, 'Plant4': 700, 'Plant5': 650, 'Plant6': 750}
# Ramp limits (up and down)
ramp_up = {'Plant1': 100, 'Plant2': 120, 'Plant3': 110, 'Plant4': 130, 'Plant5': 125, 'Plant6': 140}
ramp_down = {'Plant1': 90, 'Plant2': 110, 'Plant3': 100, 'Plant4': 120, 'Plant5': 115, 'Plant6': 130}
# Initial power output and running state
x0 = {'Plant1': 0, 'Plant2': 0, 'Plant3': 0, 'Plant4': 0, 'Plant5': 0, 'Plant6': 0}
v0 = {'Plant1': 0, 'Plant2': 0, 'Plant3': 0, 'Plant4': 0, 'Plant5': 0, 'Plant6': 0}
# Demand for each time period
d = {1: 280, 2: 327, 3: 400, 4: 388, 5: 501, 6: 600, 7: 800, 8: 927, 9: 705, 10: 502,
     11: 781, 12: 906, 13: 930, 14: 877, 15: 966}
# Create a model
model = ConcreteModel()

# Define index sets
model.I = Set(initialize=I)
model.T = Set(initialize=T)

# Define variables
model.X = Var(model.I, model.T, domain=NonNegativeReals)  # Output
model.Y = Var(model.I, model.T, domain=Binary)  # Startup
model.Z = Var(model.I, model.T, domain=Binary)  # Shutdown
model.V = Var(model.I, model.T, domain=Binary)  # Running state

# Objective function
def objective_rule(model):
    return sum(crf[i] * model.V[i, t] +
               crv[i][t-1] * model.X[i, t] +
               cu[i] * model.Y[i, t] +
               cd[i] * model.Z[i, t]
               for i in model.I for t in model.T)

model.objective = Objective(rule=objective_rule, sense=minimize)

# Constraints

# Output bounds
def output_lower_bound_rule(model, i, t):
    return l[i] * model.V[i, t] <= model.X[i, t]

def output_upper_bound_rule(model, i, t):
    return model.X[i, t] <= u[i] * model.V[i, t]

model.output_lower_bound = Constraint(model.I, model.T, rule=output_lower_bound_rule)
model.output_upper_bound = Constraint(model.I, model.T, rule=output_upper_bound_rule)

# Ramp up and down constraints
def ramp_up_rule(model, i, t):
    if t == 1:
        return model.X[i, t] - x0[i] <= ramp_up[i]
    else:
        return model.X[i, t] - model.X[i, t-1] <= ramp_up[i]

def ramp_down_rule(model, i, t):
    if t == 1:
        return x0[i] - model.X[i, t] <= ramp_down[i]
    else:
        return model.X[i, t-1] - model.X[i, t] <= ramp_down[i]

model.ramp_up = Constraint(model.I, model.T, rule=ramp_up_rule)
model.ramp_down = Constraint(model.I, model.T, rule=ramp_down_rule)

# Startup and shutdown constraints
def startup_shutdown_rule(model, i, t):
    if t == 1:
        return model.Y[i, t] - model.Z[i, t] == model.V[i, t] - v0[i]
    else:
        return model.Y[i, t] - model.Z[i, t] == model.V[i, t] - model.V[i, t-1]

model.startup_shutdown = Constraint(model.I, model.T, rule=startup_shutdown_rule)

# Demand satisfaction
def demand_rule(model, t):
    return sum(model.X[i, t] for i in model.I) == d[t]

model.demand = Constraint(model.T, rule=demand_rule)

# Reserve margin constraint
def reserve_margin_rule(model, t):
    return sum(u[i] * model.V[i, t] for i in model.I) >= 1.1 * d[t]

model.reserve_margin = Constraint(model.T, rule=reserve_margin_rule)

# Solve the model
solver = SolverFactory('gurobi')
solver.solve(model)

# Display results
print(f'Total costs are: {model.objective()}')
print()
for t in model.T:
    print(f"Time Period {t}:")
    for i in model.I:
        print(f"  {i}: Output={model.X[i, t].value}, Running={model.V[i, t].value}, Startup={model.Y[i, t].value}, Shutdown={model.Z[i, t].value}")


Total costs are: 367945.9999999996

Time Period 1:
  Plant1: Output=0.0, Running=-0.0, Startup=0.0, Shutdown=0.0
  Plant2: Output=120.0, Running=1.0, Startup=1.0, Shutdown=0.0
  Plant3: Output=105.0, Running=1.0, Startup=1.0, Shutdown=0.0
  Plant4: Output=0.0, Running=-0.0, Startup=0.0, Shutdown=0.0
  Plant5: Output=55.0, Running=1.0, Startup=1.0, Shutdown=0.0
  Plant6: Output=0.0, Running=-0.0, Startup=0.0, Shutdown=0.0
Time Period 2:
  Plant1: Output=0.0, Running=-0.0, Startup=-0.0, Shutdown=-0.0
  Plant2: Output=240.00000000000134, Running=1.0, Startup=-0.0, Shutdown=-0.0
  Plant3: Output=86.99999999999866, Running=1.0, Startup=-0.0, Shutdown=-0.0
  Plant4: Output=0.0, Running=-0.0, Startup=-0.0, Shutdown=-0.0
  Plant5: Output=0.0, Running=-0.0, Startup=-0.0, Shutdown=1.0
  Plant6: Output=0.0, Running=-0.0, Startup=-0.0, Shutdown=-0.0
Time Period 3:
  Plant1: Output=0.0, Running=-0.0, Startup=-0.0, Shutdown=-0.0
  Plant2: Output=360.0000000000051, Running=1.0, Startup=-0.0, Shutdown